# Interacting with Databases

In many applications data rarely comes from text files, that being a fairly inefficient
way to store large amounts of data. SQL-based relational databases (such as SQL Server,
PostgreSQL, and MySQL) are in wide use, and many alternative non-SQL (so-called
NoSQL) databases have become quite popular. The choice of database is usually dependent
on the performance, data integrity, and scalability needs of an application.

Loading data from SQL into a DataFrame is fairly straightforward, and pandas has
some functions to simplify the process. As an example, I’ll use an in-memory SQLite
database using Python’s built-in sqlite3 driver:

In [1]:
from pandas import DataFrame, Series

import pandas as pd

import sys

import numpy as np

import json

In [2]:
import sqlite3

In [3]:
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
c REAL, d INTEGER
);"""

In [4]:
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [5]:
# Then, insert a few rows of data:
data = [('Atlanta', 'Georgia', 1.25, 6),
('Tallahassee', 'Florida', 2.6, 3),
('Sacramento', 'California', 1.7, 5)]

stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"

con.executemany(stmt, data)

con.commit()

Most Python SQL drivers (PyODBC, psycopg2, MySQLdb, pymssql, etc.) return a list
of tuples when selecting data from a table:

In [6]:
cursor = con.execute('Select * from test')

In [7]:
rows= cursor.fetchall()

In [8]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

You can pass the list of tuples to the DataFrame constructor, but you also need the
column names, contained in the cursor’s description attribute:

In [9]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [10]:
DataFrame(rows, columns=list(zip(*cursor.description))[0])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


This is quite a bit of munging that you’d rather not repeat each time you query the
database. pandas has a read_frame function in its pandas.io.sql module that simplifies
the process. Just pass the select statement and the connection object:

In [11]:
import pandas.io.sql as sql

In [12]:
sql.sql_read_frame('select * from test', con)

AttributeError: module 'pandas.io.sql' has no attribute 'sql_read_frame'

In [ ]:
sql.read_sql_query('select * from test', con)

## Storing and Loading Data in MongoDB

NoSQL databases take many different forms. Some are simple dict-like key-value stores
like BerkeleyDB or Tokyo Cabinet, while others are document-based, with a dict-like
object being the basic unit of storage. I've chosen MongoDB (http://mongodb.org) for
my example. I started a MongoDB instance locally on my machine, and connect to it
on the default port using pymongo, the official driver for MongoDB:

In [17]:
import pymongo
#con = pymongo.Connection('localhost', port=27017)
con = pymongo.MongoClient("localhost", 27017) 


Documents stored in MongoDB are found in collections inside databases. Each running
instance of the MongoDB server can have multiple databases, and each database can
have multiple collections. Suppose I wanted to store the Twitter API data from earlier
in the chapter. First, I can access the (currently empty) tweets collection:

In [18]:
tweets = con.db.tweets

Then, I load the list of tweets and write each of them to the collection using
tweets.save (which writes the Python dict to MongoDB):

In [19]:
import requests, json

In [20]:
url = 'http://search.twitter.com/search.json?q=python%20pandas'

In [21]:
data = json.loads(requests.get(url).text)

In [23]:
for tweet in data['results']:
    tweets.save(tweet)

KeyError: 'results'

In [25]:
import datetime

In [26]:
post = {"author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()}

In [27]:
tweets.save(post)

/Users/Kevin/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  if __name__ == '__main__':


ObjectId('59011509791e4bd47d5bc08c')

In [28]:
tweets.replace_one(post)

TypeError: replace_one() missing 1 required positional argument: 'replacement'

In [29]:
tweets.insert_one(post)

DuplicateKeyError: E11000 duplicate key error collection: db.tweets index: _id_ dup key: { : ObjectId('59011509791e4bd47d5bc08c') }

Now, if I wanted to get all of my tweets (if any) from the collection, I can query the
collection with the following syntax:


In [35]:
cursor = tweets.find({'author': 'Mike'})
fields = ['author', 'text', 'tags', 'date']

# post = {"author": "Mike",
#     "text": "My first blog post!",
#     "tags": ["mongodb", "python", "pymongo"],
#     "date": datetime.datetime.utcnow()}

The cursor returned is an iterator that yields each document as a dict. As above I can
convert this into a DataFrame, optionally extracting a subset of the data fields in each
tweet:

In [36]:
result = DataFrame(list(cursor), columns=fields)

In [37]:
result

,author,text,tags,date
0,Mike,My first blog post!,"[mongodb, python, pymongo]",2017-04-26 21:44:58.209
